# Titiler expertiments
This notebook provides two possible ways to work with titiler:
- first each dataset is a separate cog thus a diferent cog endpoint
- one fat dataset with each raster as a band so we can use to `expression` in the titiler to filter on by another

In [1]:
import json
from pathlib import Path

import requests
import httpx
import branca
from folium import Map, LayerControl
from folium.raster_layers import TileLayer

titiler_endpoint = "http://127.0.0.1:8083"

In [2]:
def make_expression(filter_by: list=None, no_data=-9999):
    """Build a where expression to filters foodscapes"""
    if filter_by:
        filter_parts = "|".join([f"(b1=={x})" for x in filter_by])
        return f"where({filter_parts}, b1, {no_data});"  # the trailing ";" is critical to make it work
    else:
        return ""

def filter_with_colormap(values: list[int], cmap: dict) -> dict:
    """Changes alpha to 0 of all values in cmap except the ones in values

    Can be used as an alternative way to filter
    
    *NEVER USE!*
    """
    new_cmap = {}
    values = [str(v) for v in values]
    for k, v in cmap.items():
        if k in values:
            if len(v)==7:  # no alpha so add the alfa byte
                v = v + "FF"
            else:
                v[-2:]= "FF"
        else:
            if len(v)==7:
                v = v + "00"
            else:
                v[-2:]= "00"
        new_cmap[k] = v
    return new_cmap


def show_map(tiles):
    fig = branca.element.Figure(width="800px", height="600px")
    m = Map()
    fig.add_child(m)

    layer = TileLayer(
        tiles,
        attr="TNC",
        name="fs",
        overlay=True
    )
    m.add_child(layer)
    LayerControl().add_to(m)
    return m
    

with open("foodscapes_cmap.json") as f:
    foodscpaes_cmap = json.load(f)
    # foodscpaes_cmap = filter_with_colormap([707], foodscpaes_cmap)

## Simple cog

One COG per layer, multiple endpoints

__DON'T USE__ can't filter one raster by the value of another or mask

In [3]:
# fs_url = Path("/home/biel/Vizzuality/dev/titilerTest/demo/foodscapes/cog_tiler/cogs/Foodscapes_combinedGEOTIFF_final_COG.tif")
# paths if titiler is up with docker compose
fs_url = Path("/data/Foodscapes_combinedGEOTIFF_final_COG.tif")
risk_url = Path("/data/biodiversityrisk_COG.tif")
intensity_url = Path("/data/Foodscapes_combinedGEOTIFF_intensity_COG.tif")

In [4]:


filter_by = list(range(701, 720))
# filter_by = None

fs_r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "tile_format": "png",
        "url": fs_url,
        "colormap":json.dumps(foodscpaes_cmap),
        "return_mask":"true",
        "expression": make_expression(filter_by, -9999)  # foodscapes cog nodata value
    }
).json()

risk_r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "tile_format": "png",
        "url": risk_url,
        "colormap_name": "viridis_r",
        "expression": "where(b1>=0, b1, 0);"
    }
).json()

intensity_r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params={
        "tile_format": "png",
        "url": intensity_url,
        "colormap_name": "viridis",
        "rescale": "0,3"
    }
).json()

In [5]:
fig = branca.element.Figure(width="800px", height="600px")
m = Map()
fig.add_child(m)

fs_layer = TileLayer(
    fs_r['tiles'][0],
    attr="TNC",
    name="fs",
    overlay=True
)

risk_layer = TileLayer(
        risk_r["tiles"][0],
        attr="TNC",
        name="risk",
        overlay=True
)

intensity_layer = TileLayer(
        intensity_r["tiles"][0],
        attr="TNC",
        name="intensity",
        overlay=True
)

m.add_child(fs_layer)
m.add_child(risk_layer)
m.add_child(intensity_layer)

LayerControl().add_to(m)
m

## One single COG, multiple bands

In [6]:
# path maped into docker compose cogs/ -> /data/
stack_url = "/data/stack.tif"

In [7]:
# b1: biodiversity risk
# b2: foodscapes
# b3: intensity

# this filters foodscapes by medium intensity
expression = "where(b3==3, b2, -9999);"

r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "tile_format": "png",
        "url": stack_url,
        "colormap":json.dumps(foodscpaes_cmap),
        "expression": expression
    }
).json()

tiles = r["tiles"][0]
show_map(tiles)

## One single COG, all Risk + Foodscape bands

In [8]:
cog_url = "/data/foodscapes_stack_cog.tif"
expression = "where(b5==1, b8, -9999);"

r = httpx.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "tile_format": "png",
        "url": cog_url,
        # "colormap":json.dumps(foodscpaes_cmap),
        "colormap":json.dumps({
            0: "#FFFFFF",
            1: "#FF2A00"
        }),
        # "colormap_name":"viridis",
        # "nodata":-9999,
        # "expression": expression,
        "bidx": 1
    }
).json()

tiles = r["tiles"][0]
show_map(tiles)

In [9]:
r

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['http://127.0.0.1:8083/cog/tiles/WebMercatorQuad/{z}/{x}/{y}@1x.png?url=%2Fdata%2Ffoodscapes_stack_cog.tif&colormap=%7B%220%22%3A+%22%23FFFFFF%22%2C+%221%22%3A+%22%23FF2A00%22%7D&bidx=1'],
 'minzoom': 0,
 'maxzoom': 5,
 'bounds': [-180.0, -85.93256792029881, 180.0, 85.05112877980659],
 'center': [0.0, -0.4407195702461095, 0]}